<a href="https://colab.research.google.com/github/jaesunkkk/KOSIS-AI-competition/blob/main/KoBert_num2000_epoch7_jh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고:
https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

https://github.com/catSirup/KorEDA/blob/master/eda.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
train_data=pd.read_table('/content/drive/MyDrive/통계청 기술분류/데이터/1. 실습용자료.txt',sep='|',encoding='cp949')

In [ ]:
test_data=pd.read_table('/content/drive/MyDrive/통계청 기술분류/데이터/2. 모델개발용자료.txt',sep='|',encoding='cp949')

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook,notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

#토크나이저
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

#명사추출
!pip install konlpy
from konlpy.tag import Okt

#불균형 처리
import pickle
import re

import random

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.6
    Uninstalling tokenizers-0.11.6:
      Successfully uninstalled tokenizers-0.11.6
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers>=4.8.1, but you have transformers 3.0.2 which is incompatible.
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-e4vkrnon
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-e4vkrnon
  Using cached transformers-4

In [ ]:
train_data.isna().sum()

AI_id            0
digit_1          0
digit_2          0
digit_3          0
text_obj     16677
text_mthd    43619
text_deal    67652
dtype: int64

In [ ]:
train_data['text_obj']=train_data['text_obj'].fillna(' ')
train_data['text_mthd']=train_data['text_mthd'].fillna(' ')
train_data['text_deal']=train_data['text_deal'].fillna(' ')

In [ ]:
test_data['text_obj']=test_data['text_obj'].fillna(' ')
test_data['text_mthd']=test_data['text_mthd'].fillna(' ')
test_data['text_deal']=test_data['text_deal'].fillna(' ')

In [ ]:
train_data['text']=train_data['text_obj']+" "+train_data['text_mthd']+" "+train_data['text_deal']
test_data['text']=test_data['text_obj']+' '+test_data['text_mthd']+' '+test_data['text_deal']

중복제거

In [ ]:
train_data.text.nunique()

699218

In [ ]:
train_data.drop_duplicates(subset='text',inplace=True)

In [ ]:
train_data.shape

(699218, 8)

digit_3 레이블당 최대 2000개 뽑기

In [ ]:
num=2000

In [ ]:
import random

In [ ]:
digit_3=list(train_data['digit_3'].unique())
len(digit_3)

225

In [ ]:
numbers_by_digit_3=pd.DataFrame(train_data.groupby('digit_3').count().AI_id)

In [ ]:
put_all=list(numbers_by_digit_3.loc[numbers_by_digit_3.AI_id<num,:].index)

In [ ]:
len(put_all)

156

In [ ]:
train_data.digit_3.nunique()

225

In [ ]:
dict={}
for i in range(train_data.digit_3.nunique()):
  dict[digit_3[i]]=i
dict

{11: 174,
 12: 197,
 14: 188,
 20: 171,
 31: 199,
 32: 179,
 51: 223,
 61: 209,
 62: 207,
 71: 121,
 72: 80,
 80: 218,
 101: 97,
 102: 10,
 103: 88,
 104: 100,
 105: 135,
 106: 72,
 107: 7,
 108: 178,
 111: 148,
 112: 110,
 120: 224,
 131: 158,
 132: 107,
 133: 177,
 134: 103,
 139: 111,
 141: 43,
 142: 215,
 143: 180,
 144: 122,
 151: 91,
 152: 182,
 161: 95,
 162: 115,
 163: 67,
 171: 108,
 172: 138,
 179: 129,
 181: 81,
 182: 204,
 191: 195,
 192: 203,
 201: 151,
 202: 90,
 203: 56,
 204: 79,
 205: 157,
 211: 206,
 212: 187,
 213: 201,
 221: 18,
 222: 33,
 231: 156,
 232: 120,
 233: 94,
 239: 85,
 241: 130,
 242: 104,
 243: 69,
 251: 87,
 252: 198,
 259: 28,
 261: 63,
 262: 65,
 263: 113,
 264: 149,
 265: 152,
 266: 222,
 271: 142,
 272: 141,
 273: 193,
 274: 162,
 281: 52,
 282: 196,
 283: 166,
 284: 114,
 285: 146,
 289: 131,
 291: 5,
 292: 20,
 301: 202,
 302: 145,
 303: 116,
 304: 217,
 311: 77,
 312: 192,
 313: 185,
 319: 167,
 320: 35,
 331: 123,
 332: 194,
 333: 169,
 334: 15

In [ ]:
quarter_train=pd.DataFrame(columns=train_data.columns)
for i in digit_3:
  if i in put_all:
    x=train_data.loc[train_data.digit_3.isin([i]),:]
  else:
    index=random.sample(list(train_data.loc[train_data.digit_3.isin([i]),:].index),num)
    x=train_data.loc[index,:]
  quarter_train=quarter_train.append(x)
  

In [ ]:
quarter_train.shape # 각 레이블 마다 자료 1000개 이하로 존재. 데이터 프레임 형태

(225323, 8)

In [ ]:
quarter_train.reset_index(drop=True,inplace=True)

In [ ]:
quarter_train

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0264906,S,95,952,카센타에서,자동차특정부분 수리,타어어교환 경정비,카센타에서 자동차특정부분 수리 타어어교환 경정비
1,id_0041203,S,95,952,사업장에서,고객요청으로,탑차수리,사업장에서 고객요청으로 탑차수리
2,id_0135743,S,95,952,정비소에서,고객을 대상으로,자동차수리,정비소에서 고객을 대상으로 자동차수리
3,id_0343538,S,95,952,특수유리를,자동차소지고객 요구에따라,차량용유리교체 서비스,특수유리를 자동차소지고객 요구에따라 차량용유리교체 서비스
4,id_0471332,S,95,952,매장에서,자동차외관,왁스코팅,매장에서 자동차외관 왁스코팅
...,...,...,...,...,...,...,...,...
225318,id_0950557,C,26,266,"금속,테잎용원단",가공,마그네틱테이프,"금속,테잎용원단 가공 마그네틱테이프"
225319,id_0436665,B,5,51,무연탄,"발파, 생산",,"무연탄 발파, 생산"
225320,id_0756750,C,12,120,"잎담배, 판상엽","가공, 궐련","차세대상품, 차세대제조담배","잎담배, 판상엽 가공, 궐련 차세대상품, 차세대제조담배"
225321,id_0904129,C,12,120,향료,"재료투입,정제,혼합,출하",전자담배 액상,"향료 재료투입,정제,혼합,출하 전자담배 액상"


명사 추출

In [ ]:
def noun_parsing(contents):
  okt=Okt()
  noun_data=' '.join(okt.nouns(contents))
  return noun_data

In [ ]:
tqdm.pandas()
quarter_train["text"]=quarter_train['text'].progress_apply(noun_parsing)

100%|██████████| 225323/225323 [10:15<00:00, 365.79it/s]


In [ ]:
quarter_train

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0264906,S,95,952,카센타에서,자동차특정부분 수리,타어어교환 경정비,카 타 자동차 특정 부분 수리 어어 교환 정비
1,id_0041203,S,95,952,사업장에서,고객요청으로,탑차수리,사업 고객 요청 탑차 수리
2,id_0135743,S,95,952,정비소에서,고객을 대상으로,자동차수리,정비 소 고객 대상 자동차 수리
3,id_0343538,S,95,952,특수유리를,자동차소지고객 요구에따라,차량용유리교체 서비스,특수 유리 자동차 소지 고객 요구 차량 용 유리 교체 서비스
4,id_0471332,S,95,952,매장에서,자동차외관,왁스코팅,매장 자동차 외관 왁스 코팅
...,...,...,...,...,...,...,...,...
225318,id_0950557,C,26,266,"금속,테잎용원단",가공,마그네틱테이프,금속 테잎 용 원단 가공 마 그네 테이프
225319,id_0436665,B,5,51,무연탄,"발파, 생산",,무연탄 발파 생산
225320,id_0756750,C,12,120,"잎담배, 판상엽","가공, 궐련","차세대상품, 차세대제조담배",잎담배 판상엽 가공 궐련 차세대 상품 차세대 제조 담배
225321,id_0904129,C,12,120,향료,"재료투입,정제,혼합,출하",전자담배 액상,향료 재료 투입 정제 혼합 출하 전자담배 액


text 내에서 중복되는 명사 제거

In [ ]:
def deletion(data):
  for i in tqdm(range(data.shape[0])):
    result=[]
    temp=word_tokenize(data.loc[i,'text'])
    for j in temp:
      if j not in result:
        result.append(j)
    data.loc[i,'text']=' '.join(result)

In [ ]:
deletion(quarter_train)

100%|██████████| 225323/225323 [32:26<00:00, 115.74it/s]


In [ ]:
quarter_train

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0264906,S,95,952,카센타에서,자동차특정부분 수리,타어어교환 경정비,카 타 자동차 특정 부분 수리 어어 교환 정비
1,id_0041203,S,95,952,사업장에서,고객요청으로,탑차수리,사업 고객 요청 탑차 수리
2,id_0135743,S,95,952,정비소에서,고객을 대상으로,자동차수리,정비 소 고객 대상 자동차 수리
3,id_0343538,S,95,952,특수유리를,자동차소지고객 요구에따라,차량용유리교체 서비스,특수 유리 자동차 소지 고객 요구 차량 용 교체 서비스
4,id_0471332,S,95,952,매장에서,자동차외관,왁스코팅,매장 자동차 외관 왁스 코팅
...,...,...,...,...,...,...,...,...
225318,id_0950557,C,26,266,"금속,테잎용원단",가공,마그네틱테이프,금속 테잎 용 원단 가공 마 그네 테이프
225319,id_0436665,B,5,51,무연탄,"발파, 생산",,무연탄 발파 생산
225320,id_0756750,C,12,120,"잎담배, 판상엽","가공, 궐련","차세대상품, 차세대제조담배",잎담배 판상엽 가공 궐련 차세대 상품 제조 담배
225321,id_0904129,C,12,120,향료,"재료투입,정제,혼합,출하",전자담배 액상,향료 재료 투입 정제 혼합 출하 전자담배 액


In [ ]:
quarter_train.to_csv('/content/drive/MyDrive/통계청/2000개명사제거.csv',index=False)

불균형 처리

In [ ]:
#quarter_train=pd.read_table('/content/drive/MyDrive/통계청/1000개명사제거.csv',sep=',')

In [ ]:
quarter_train=quarter_train[['digit_3','text']]

In [ ]:
quarter_train['text']=quarter_train['text'].str.split(' ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
quarter_train

,digit_3,text
0,952,"[카, 타, 자동차, 특정, 부분, 수리, 어어, 교환, 정비]"
1,952,"[사업, 고객, 요청, 탑차, 수리]"
2,952,"[정비, 소, 고객, 대상, 자동차, 수리]"
3,952,"[특수, 유리, 자동차, 소지, 고객, 요구, 차량, 용, 교체, 서비스]"
4,952,"[매장, 자동차, 외관, 왁스, 코팅]"
...,...,...
225318,266,"[금속, 테잎, 용, 원단, 가공, 마, 그네, 테이프]"
225319,51,"[무연탄, 발파, 생산]"
225320,120,"[잎담배, 판상엽, 가공, 궐련, 차세대, 상품, 제조, 담배]"
225321,120,"[향료, 재료, 투입, 정제, 혼합, 출하, 전자담배, 액]"


In [ ]:
import pickle
import re

In [ ]:
digit_count=quarter_train.digit_3.value_counts().to_dict()

In [ ]:
def random_swap(words, digit_3):
  n=2
  new_words=words.copy()
  for _ in range(n):
      new_words=swap_word(new_words)
      data={'digit_3':digit_3,'text':new_words}
  return data

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

def iteration(dataframe):
  y=pd.DataFrame([],columns=['digit_3','text'])
  for i in tqdm(range(dataframe.shape[0])):
      for _ in range(int((num-digit_count[dataframe.loc[i,'digit_3']])/digit_count[dataframe.loc[i,'digit_3']])):
          y=y.append(random_swap(dataframe.loc[i,'text'],dataframe.loc[i,'digit_3']),ignore_index=True)
  return y


In [ ]:
temp=iteration(quarter_train)

100%|██████████| 225323/225323 [10:53<00:00, 344.66it/s]


In [ ]:
temp.shape

(187838, 2)

In [ ]:
temp.shape[0]+quarter_train.shape[0]

413161

In [ ]:
balanced=quarter_train.append(temp)

In [ ]:
balanced.shape # 대충 225000개 정도 되야함.

(413161, 2)

In [ ]:
balanced.reset_index(inplace=True,drop=True)

In [ ]:
balanced

,digit_3,text
0,952,"[카, 타, 자동차, 특정, 부분, 수리, 어어, 교환, 정비]"
1,952,"[사업, 고객, 요청, 탑차, 수리]"
2,952,"[정비, 소, 고객, 대상, 자동차, 수리]"
3,952,"[특수, 유리, 자동차, 소지, 고객, 요구, 차량, 용, 교체, 서비스]"
4,952,"[매장, 자동차, 외관, 왁스, 코팅]"
...,...,...
413156,120,"[담배잎, 원료, 투입, 줄기, 슬러리, 제조]"
413157,120,"[담배잎, 줄기, 슬러리, 제조, 원료, 투입]"
413158,120,"[제조, 줄기, 원료, 투입, 담배잎, 슬러리]"
413159,120,"[원료, 줄기, 슬러리, 투입, 담배잎, 제조]"


In [ ]:
quarter_train.to_csv('/content/drive/MyDrive/통계청/2000개명사제거+balanced.csv',index=False)

In [ ]:
balanced['text']=balanced['text'].str.join(' ')

In [ ]:
balanced

,digit_3,text
0,952,카 타 자동차 특정 부분 수리 어어 교환 정비
1,952,사업 고객 요청 탑차 수리
2,952,정비 소 고객 대상 자동차 수리
3,952,특수 유리 자동차 소지 고객 요구 차량 용 교체 서비스
4,952,매장 자동차 외관 왁스 코팅
...,...,...
413156,120,담배잎 원료 투입 줄기 슬러리 제조
413157,120,담배잎 줄기 슬러리 제조 원료 투입
413158,120,제조 줄기 원료 투입 담배잎 슬러리
413159,120,원료 줄기 슬러리 투입 담배잎 제조


임베딩 

In [ ]:
data_list=[]
for text,label in zip(balanced['text'],balanced['digit_3']):
      data=[]
      data.append(text)
      data.append(dict[label])

      data_list.append(data)

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

330528
82633


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 32
batch_size = 64
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 2431,  767, 2600, 2614, 3829,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
       dtype=int32),
 array(7, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 57)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=225,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
#from sklearn.metrics import f1_score
#true=list(train_data['digit_3'])

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
       
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/5165 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 5.499617576599121 train acc 0.015625
epoch 1 batch id 201 loss 5.372530937194824 train acc 0.0080068407960199
epoch 1 batch id 401 loss 4.924640655517578 train acc 0.025015586034912718
epoch 1 batch id 601 loss 4.400680065155029 train acc 0.09250207986688852
epoch 1 batch id 801 loss 3.7757105827331543 train acc 0.18865090511860175
epoch 1 batch id 1001 loss 3.6030728816986084 train acc 0.27673888611388614
epoch 1 batch id 1201 loss 3.1301965713500977 train acc 0.3498126561199001
epoch 1 batch id 1401 loss 2.7627639770507812 train acc 0.40763294075660245
epoch 1 batch id 1601 loss 2.011545181274414 train acc 0.4533592286071205
epoch 1 batch id 1801 loss 1.7907661199569702 train acc 0.49082974736257634
epoch 1 batch id 2001 loss 1.294047236442566 train acc 0.5221061344327836
epoch 1 batch id 2201 loss 1.329153060913086 train acc 0.5484367900954111
epoch 1 batch id 2401 loss 0.6827741265296936 train acc 0.5712398479800084
epoch 1 batch id 2601 loss 1.0693330764770

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1292 [00:00<?, ?it/s]

...

epoch 6 test acc 0.9212098490712074


  0%|          | 0/5165 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.06456644088029861 train acc 0.984375
epoch 7 batch id 201 loss 0.05103721097111702 train acc 0.9661847014925373
epoch 7 batch id 401 loss 0.050613339990377426 train acc 0.9663341645885287
epoch 7 batch id 601 loss 0.11543559283018112 train acc 0.9659941763727121
epoch 7 batch id 801 loss 0.15079116821289062 train acc 0.9670138888888888
epoch 7 batch id 1001 loss 0.11635876446962357 train acc 0.9671266233766234
epoch 7 batch id 1201 loss 0.11141762137413025 train acc 0.967331910907577
epoch 7 batch id 1401 loss 0.08364216238260269 train acc 0.9671328515346181
epoch 7 batch id 1601 loss 0.04039066284894943 train acc 0.9671299188007495
epoch 7 batch id 1801 loss 0.019992975518107414 train acc 0.9673705580233204
epoch 7 batch id 2001 loss 0.12519966065883636 train acc 0.9677348825587206
epoch 7 batch id 2201 loss 0.04134687781333923 train acc 0.9680613925488414
epoch 7 batch id 2401 loss 0.03666796162724495 train acc 0.9683465222823824
epoch 7 batch id 2601 loss 0

  0%|          | 0/1292 [00:00<?, ?it/s]

epoch 7 test acc 0.9223103715170279


In [ ]:
tqdm.pandas()
test_data['text']=test_data['text'].progress_apply(noun_parsing)

100%|██████████| 100000/100000 [06:57<00:00, 239.43it/s]


In [ ]:
deletion(test_data)

100%|██████████| 100000/100000 [04:56<00:00, 337.39it/s]


In [ ]:
test_list=[]

for text in test_data['text']:
    data=[]
    data.append(text)
    data.append(0)

    test_list.append(data)

In [ ]:
test_data

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매,치킨 전문점 고객 주문 해 판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구,산업 공구 다른 소매업자 철물
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영,절 대상 불교 단체 운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝,영업 고객 요구 자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공",실내 포장마차 접객 시설 소주 맥주 제공
...,...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장,사업 일반인 대상 버섯 농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료,한의원 외래 환자 위주 치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매,점포 소비자 그림 판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공,사업 일반인 학생 대상 학습 공간 제공


In [ ]:
data_test = BERTDataset(test_list, 0, 1, tok, max_len, True, False)
real_test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
model.eval()


result = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(real_test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()
      final = np.argmax(logits)
  result.append(final)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/100000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
len(result)

100000

In [ ]:
dict_prediction=pd.DataFrame(result,columns=['dict_predict'])

In [ ]:
dict_prediction.to_csv('/content/drive/MyDrive/통계청/dict_predict4(kobert_2000_balance).csv',header=True, index=False)

In [ ]:
result=pd.read_table('/content/drive/MyDrive/통계청/dict_predict4(kobert_2000_balance).csv',sep=',')

In [ ]:
dict_reverse={}

for key, value in dict.items():
  dict_reverse[value]=key

In [ ]:
for i in tqdm(range(100000)):
  test_data.loc[i,'digit_3']=dict_reverse[result.loc[i,'dict_predict']]

100%|██████████| 100000/100000 [01:08<00:00, 1457.60it/s]


In [ ]:
test_data=test_data.astype({'digit_3':'int'})

In [ ]:
test_data['digit_2']=(test_data['digit_3']/10).astype(int)

In [ ]:
# digit_1,digit_2 연결 사전
list=[]

for digit_1,digit_2 in zip(train_data['digit_1'],train_data['digit_2']):
  data=[]
  data.append(digit_1)
  data.append(digit_2)
  list.append(data)

new_list=[]

for i in list:
  if i not in new_list:
    new_list.append(i)



In [ ]:
dict_digit1_digit2={}

In [ ]:
for i in new_list:
  dict_digit1_digit2[i[1]]=i[0]

In [ ]:
for i in tqdm(range(test_data.shape[0])):
  test_data.loc[i,'digit_1']=dict_digit1_digit2[test_data.loc[i,'digit_2']]

100%|██████████| 100000/100000 [02:27<00:00, 679.05it/s]


In [ ]:
test_data.drop('text',inplace=True,axis=1)

In [ ]:
test_data.to_csv('/content/drive/MyDrive/통계청/kobert_num2000.csv',index=False)